In [1]:
!pip install pandas | tail -n 1
!pip install geopandas | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install shapely | tail -n 1

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import sys
import math
import os
import json
from sklearn.neighbors import NearestNeighbors
from shapely.geometry import Point, Polygon
from scipy.spatial import Voronoi
from google.colab import files
import warnings

# 将来の警告を非表示にする
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
print("分析対象のCSVファイルをアップロードしてください...")
uploaded = files.upload()
file_name = next(iter(uploaded))
print(f"'{file_name}' をアップロードしました。")

分析対象のCSVファイルをアップロードしてください...


Saving 4-2-05-syuuroukeizoku-A.csv to 4-2-05-syuuroukeizoku-A.csv
'4-2-05-syuuroukeizoku-A.csv' をアップロードしました。


In [16]:
# --- 1. データの読み込みと準備 ---

# ◆ 対象施設データをCSVから読み込み
try:
    print("放課後等デイサービス事業所のデータを読み込んでいます...")
    df = pd.read_csv(file_name, encoding='cp932')

    # 必要な列を抽出・リネーム
    df = df[['事業所名', '所在地', '緯度', '経度']].rename(columns={
        '事業所名': 'name',
        '所在地': 'address',
        '緯度': 'lat',
        '経度': 'lon'
    })

    # 緯度・経度が数値でない行や欠損値を除外
    df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
    df['lon'] = pd.to_numeric(df['lon'], errors='coerce')
    df.dropna(subset=['lat', 'lon'], inplace=True)
    df = df.reset_index(drop=True).reset_index().rename(columns={'index': 'id'}) # IDを付与

    print(f"対象施設を読み込みました。施設数: {len(df)}件")

except Exception as e:
    print(f"データの読み込みまたは処理に失敗しました: {e}", file=sys.stderr)
    sys.exit(1)

# ◆ 東京都の境界線データを準備
try:
    print("東京都の境界線データを準備しています...")
    geo_url = "https://raw.githubusercontent.com/dataofjapan/land/master/tokyo.geojson"
    gdf_tokyo_all = gpd.read_file(geo_url)

    islands = ['大島町', '利島村', '新島村', '神津島村', '三宅村', '御蔵島村', '八丈町', '青ヶ島村', '小笠原村']
    gdf_tokyo = gdf_tokyo_all[~gdf_tokyo_all['ward_ja'].isin(islands)].copy()

    tokyo_boundary_poly = gdf_tokyo.unary_union
    print("境界線データの準備が完了しました。")
except Exception as e:
    print(f"境界線データの準備に失敗しました: {e}", file=sys.stderr)
    sys.exit(1)

# --- 2. データのエクスポート処理 ---

# ◆ 出力用ディレクトリを作成
output_dir = 'output_data'
os.makedirs(output_dir, exist_ok=True)
print(f"'{output_dir}/' ディレクトリを作成しました。")

# ◆ ファイル1: 市区町村境界データ (municipalities.geojson)
municipalities_path = os.path.join(output_dir, 'municipalities.geojson')
gdf_tokyo.to_file(municipalities_path, driver='GeoJSON')
print(f"✅ 市区町村データを '{municipalities_path}' に保存しました。")

# ◆ ファイル2: 施設データ (facilities.json)
facilities_path = os.path.join(output_dir, 'facilities.json')
df.to_json(facilities_path, orient='records', force_ascii=False, indent=2)
print(f"✅ 施設データを '{facilities_path}' に保存しました。")

# ◆ ファイル3: アクセス距離メッシュデータ (mesh.geojson)
print("アクセス距離メッシュを計算しています...")
mesh_size_m = 250
min_lon, min_lat, max_lon, max_lat = gdf_tokyo.total_bounds
center_lat_tokyo = tokyo_boundary_poly.centroid.y
m_per_deg_lat = 111320.0
m_per_deg_lon = m_per_deg_lat * math.cos(math.radians(center_lat_tokyo))
step_lat, step_lon = mesh_size_m / m_per_deg_lat, mesh_size_m / m_per_deg_lon
grid_lons, grid_lats = np.arange(min_lon, max_lon, step_lon), np.arange(min_lat, max_lat, step_lat)

facilities_rad = np.radians(df[['lat', 'lon']].to_numpy())
nn_model = NearestNeighbors(n_neighbors=1, algorithm='ball_tree', metric='haversine').fit(facilities_rad)

mesh_features = []
for lat in grid_lats:
    for lon in grid_lons:
        point = Point(lon, lat)
        if tokyo_boundary_poly.contains(point):
            point_rad = np.radians([[lat, lon]])
            distance_rad, _ = nn_model.kneighbors(point_rad)
            distance_m = distance_rad[0][0] * 6371000

            half_step_lat, half_step_lon = step_lat / 2, step_lon / 2
            min_p_lon, min_p_lat = lon - half_step_lon, lat - half_step_lat
            max_p_lon, max_p_lat = lon + half_step_lon, lat + half_step_lat

            mesh_polygon = Polygon([
                (min_p_lon, min_p_lat), (max_p_lon, min_p_lat),
                (max_p_lon, max_p_lat), (min_p_lon, max_p_lat),
                (min_p_lon, min_p_lat)
            ])
            mesh_features.append({'geometry': mesh_polygon, 'distance_m': distance_m})

if mesh_features:
    mesh_gdf = gpd.GeoDataFrame(mesh_features, geometry='geometry', crs="EPSG:4326")
    mesh_path = os.path.join(output_dir, 'mesh.geojson')
    mesh_gdf.to_file(mesh_path, driver='GeoJSON')
    print(f"✅ アクセス距離メッシュデータを '{mesh_path}' に保存しました。")

# ◆ ファイル4: ボロノイ領域データ (voronoi.geojson)
print("ボロノイ領域を計算しています...")
def voronoi_finite_polygons_2d(vor, radius=None):
    if vor.points.shape[1] != 2: raise ValueError("Requires 2D input")
    new_regions, new_vertices = [], vor.vertices.tolist()
    center = vor.points.mean(axis=0)
    if radius is None: radius = np.ptp(vor.points, axis=0).max() * 2
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        if all(v >= 0 for v in vertices):
            new_regions.append(vertices)
            continue
        ridges = all_ridges.get(p1, [])
        if not ridges: continue
        new_region = [v for v in vertices if v >= 0]
        for p2, v1, v2 in ridges:
            if v2 < 0: v1, v2 = v2, v1
            if v1 >= 0: continue
            t = vor.points[p2] - vor.points[p1]; t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])
            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius
            new_vertices.append(far_point.tolist())
            new_region.append(len(new_vertices) - 1)
        vs = np.asarray([new_vertices[v] for v in new_region])
        c, angles = vs.mean(axis=0), np.arctan2(vs[:, 1] - vs.mean(axis=0)[1], vs[:, 0] - vs.mean(axis=0)[0])
        new_region = np.array(new_region)[np.argsort(angles)]
        new_regions.append(new_region.tolist())
    return new_regions, np.asarray(new_vertices)

if len(df) >= 4:
    points_for_voronoi = df[['lon', 'lat']].to_numpy()
    vor = Voronoi(points_for_voronoi)
    finite_regions, finite_vertices = voronoi_finite_polygons_2d(vor)

    voronoi_polygons = []
    for region in finite_regions:
        polygon_vertices = finite_vertices[region]
        voronoi_poly = Polygon(polygon_vertices)
        clipped_poly = voronoi_poly.intersection(tokyo_boundary_poly)
        if not clipped_poly.is_empty:
            voronoi_polygons.append(clipped_poly)

    voronoi_gdf = gpd.GeoDataFrame(geometry=voronoi_polygons, crs="EPSG:4326")
    voronoi_path = os.path.join(output_dir, 'voronoi.geojson')
    voronoi_gdf.to_file(voronoi_path, driver='GeoJSON')
    print(f"✅ ボロノイ領域データを '{voronoi_path}' に保存しました。")

print(f"\n--- 全ての処理が完了しました ---")
print(f"'{output_dir}' フォルダ内に4つのデータファイルが生成されました。")
print("左のファイルパネルからフォルダをzip圧縮してダウンロードできます。")

放課後等デイサービス事業所のデータを読み込んでいます...
対象施設を読み込みました。施設数: 104件
東京都の境界線データを準備しています...


/tmp/ipython-input-1652260482.py:37: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  tokyo_boundary_poly = gdf_tokyo.unary_union


境界線データの準備が完了しました。
'output_data/' ディレクトリを作成しました。
✅ 市区町村データを 'output_data/municipalities.geojson' に保存しました。
✅ 施設データを 'output_data/facilities.json' に保存しました。
アクセス距離メッシュを計算しています...
✅ アクセス距離メッシュデータを 'output_data/mesh.geojson' に保存しました。
ボロノイ領域を計算しています...
✅ ボロノイ領域データを 'output_data/voronoi.geojson' に保存しました。

--- 全ての処理が完了しました ---
'output_data' フォルダ内に4つのデータファイルが生成されました。
左のファイルパネルからフォルダをzip圧縮してダウンロードできます。
